In [33]:
%pylab
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import os
import pandas as pd

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [34]:
data_file = './data_electrico/VBS__052.VBO'
with open(data_file, 'r',encoding='cp1252') as vboxdata:
    section = []
    time = []
    lat = []
    long = []
    vel = []
    height = []
    vertv = []
    for line in vboxdata:
        if line[0] == '[':
            section = str.replace(line, "\n","")
            continue
        if section == '[column names]':
            col_names = str.replace(line, "  \n","").split('  ')
            section = []
        if section == '[data]':
            linesplit = str.replace(line, " \n","").split(' ')
            time.append(float(linesplit[1][0:2])*3600+float(linesplit[1][2:4])*60+float(linesplit[1][4:]))
            lat.append(float(linesplit[2]))
            long.append(float(linesplit[3]))
            vel.append(float(linesplit[4]))
            height.append(float(linesplit[6]))
            vertv.append(float(linesplit[7]))

time = np.asarray(time)
lat = np.asarray(lat)
long = np.asarray(long)
vel = np.asarray(vel)
height = np.asarray(height)
vertv = np.asarray(vertv)
time_base = time[0]
time = time - time_base
lat = lat/60.0
long = -long/60.0
dt = time[1]-time[0]

In [56]:
datos = pd.DataFrame(columns=['Tiempo','Lat','Long','Velocidad','Height','Vertv'])
datos['Tiempo']=time
datos['Lat']=lat
datos['Long']=long
datos['Velocidad']=vel
datos['Height']=height
datos['Vertv']=vertv